In [1]:
import json
from abc import ABCMeta, abstractmethod

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [4]:
from train.train_debug import *

In [5]:
Z, X1, X2, Y1, Y2, params, preprocess_info = load_training_data('../training_data/example_query/','q0')

In [6]:
feature_generator = FeatureGenerator()

In [7]:
feature_generator.fit(X1 + X2)

RelType :  {'Seq Scan', 'Nested Loop', 'Index Scan', 'Hash Join', 'Hash'}


In [8]:
vars(feature_generator.normalizer)

{'_mins': {'Startup Cost': 0.0,
  'Total Cost': 0.371563556432483,
  'Plan Rows': 0.6931471805599453},
 '_maxs': {'Startup Cost': 13.564519206060003,
  'Total Cost': 13.673290407169015,
  'Plan Rows': 17.404826908902354}}

In [9]:
vars(feature_generator.feature_parser)

{'normalizer': <model.feature.Normalizer at 0x22c1a597e48>,
 'input_relations': ['char_name', 'aka_title', 'name', 'title', 'cast_info']}

In [10]:
vars(feature_generator.feature_parser.normalizer)

{'_mins': {'Startup Cost': 0.0,
  'Total Cost': 0.371563556432483,
  'Plan Rows': 0.6931471805599453},
 '_maxs': {'Startup Cost': 13.564519206060003,
  'Total Cost': 13.673290407169015,
  'Plan Rows': 17.404826908902354}}

In [11]:
len(X1), len(Z)

(9, 9)

In [12]:
list(zip(*Z))

[('2014', '2014', '2014', '2013', '2013', '2013', '2013', '2013', '2013'),
 ('%John%',
  '%John%',
  '%John%',
  '%smith%',
  '%smith%',
  '%smith%',
  '%John%',
  '%John%',
  '%John%')]

In [13]:
X1_ = feature_generator.transform(X1)
X2_ = feature_generator.transform(X2)
Z_ = feature_generator.transform_z(Z, params, preprocess_info)

In [14]:
Z_

array([[34, 34, 34, 33, 33, 33, 33, 33, 33],
       [ 0,  0,  0,  1,  1,  1,  0,  0,  0]])

In [15]:
vector = list(zip(*Z))
processed_components = []
for i, (param, preprocessing_info) in enumerate(zip(params, preprocess_info)):
    data_type = param["data_type"]
    preprocessing_type = preprocessing_info["type"]
    layer = vector[i]
    if data_type == "float" and preprocessing_type == "std_normalization":
        mean = preprocessing_info["mean"]
        std = torch.sqrt(preprocessing_info["variance"])
        processed_components.append((np.array(layer).astype(int) - mean) / std)
    elif data_type == "int":
        shifted_layer = np.array(layer).astype(int) - param["min"]
        processed_components.append(shifted_layer)
    elif data_type == "text":
        vocab = {word: idx for idx, word in enumerate(param["distinct_values"])}
        num_oov_indices = preprocessing_info.get("num_oov_indices", 0)
        lookup_layer = np.array([vocab.get(la, len(vocab)) for la in layer])
        processed_components.append(lookup_layer)
        
processed_components

[array([34, 34, 34, 33, 33, 33, 33, 33, 33]),
 array([0, 0, 0, 1, 1, 1, 0, 0, 0])]

In [16]:
rank_PQO_model = RankPQOModel(feature_generator, 'q0', preprocess_info)

In [17]:
rank_PQO_model